In [2]:
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset

In [3]:
train_data_path = '/opt/ml/code/df/train/df_train_20.csv'
valid_data_path = '/opt/ml/code/df/valid/df_valid_20.csv'
df_train = pd.read_csv(train_data_path)
df_valid = pd.read_csv(valid_data_path)

In [28]:
def expand_mask(df):

    masks = ['normal', 'incorrect', 'mask1', 'mask2', 'mask3', 'mask4', 'mask5']
    d = {f:[] for f in masks}
    for _, row in df.iterrows():
        files = os.listdir(os.path.join('/opt/ml/input/data/train/images', row.path))
        for field in d:
            for file in files:
                if file.startswith(field):
                    d[field].append(os.path.join('/opt/ml/input/data/train/images', row.path, file))
                    break
            else:
                raise FileNotFoundError
    for f in d:
        df[f] = d[f]
    
    return df

In [4]:
def generate_mask_field(df):
    
    def _get_extension(row):
        for f in os.listdir(os.path.join('/opt/ml/input/data/train/images', row.path)):
            if f.startswith(row['mask']):
                return os.path.join('/opt/ml/input/data/train/images', row.path, f)
        raise FileNotFoundError

    mask_map = {'incorrect_mask': 1, 'mask1': 0, 'mask2': 0, 'mask3': 0,
                'mask4': 0, 'mask5': 0, 'normal': 2}   

    df['gender'] = df['gender'].map({'male':0, 'female':1})
    df['id'] = list(range(len(df)))
    df['mask'] = [['incorrect_mask', 'mask1', 'mask2', 'mask3', 'mask4', 'mask5', 'normal'] for _ in range(len(df))]
                    
    df = df.explode('mask')
    df['path'] = df.apply(_get_extension, axis=1)
    df['mask'] = df['mask'].map(mask_map)

    return df

In [5]:
df_key = generate_mask_field(df_train)

In [7]:
df_key.to_csv('../df/df_key_cluster.csv', index=False)

In [30]:
df_train = expand_mask(df_train)

In [31]:
df_valid = generate_mask_field(df_valid)

In [32]:
df_train.to_csv('../df/df_train_cluster.csv', index=False)
df_valid.to_csv('../df/df_valid_cluster.csv', index=False)